# W-net Model - Train

In [ ]:
%pip install tensorflow

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import os
import glob
import sys
from keras.optimizers import  Adam


# Importing our w-net model
MY_UTILS_PATH = "../Modules/"
if not MY_UTILS_PATH in sys.path:
    sys.path.append(MY_UTILS_PATH)
import frequency_spatial_network as fsnet

# Importing callbacks and data augmentation utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import keras
print(keras.__version__)

print(keras.__version__)

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
print(fsnet.wnet_new(1,2,3,4))

In [ ]:
from google.cloud import storage
client = storage.Client()
BUCKET_NAME = 'calgary_iitrpr'
elements = client.list_blobs(BUCKET_NAME)
files=[a.name for a in elements]
Trains = [x for x in files if "TrainData" in x and "npy" in x]
print(Trains)

In [ ]:
# Train Set 
train_path = r"GCS:calgary_iitrpr/calgary/TrainData/TrainData/*.npy"
# train_path = r"D:\Dataset\calgary\Train\Train\e14141s3_P58880.7.npy"
# kspace_files_train = np.asarray(glob.glob(train_path))
kspace_files_train = np.asarray([r"gs://calgary_iitrpr/"+x for x in files if "TrainData" in x and "npy" in x])

# Validation set
val_path = r"gs://calgary_iitrpr/calgary/Test/Test/*.npy"
# val_path = r"D:\Dataset\calgary\Val\Val\e14351s3_P29184.7.npy"
kspace_files_val = np.asarray([r"gs://calgary_iitrpr/"+x for x in files if "Val" in x and "npy" in x])

indexes = np.arange(kspace_files_train.size,dtype = int)
np.random.shuffle(indexes)
kspace_files_train = kspace_files_train[indexes]


print(kspace_files_train[-1])
print(len(kspace_files_train))

print(kspace_files_val[-1])
print(len(kspace_files_val))

In [ ]:
from io import BytesIO
from tensorflow.python.lib.io import file_io

In [ ]:
under_rate = '20'
imshape = (256,256)
norm = np.sqrt(imshape[0]*imshape[1])
nchannels = 2 #complex data real + imag

# undersampling patterns - uncentred k-space
var_sampling_mask = np.load("../Data/sampling_mask_" + under_rate + "perc.npy")

print("Undersampling:", 1.0*var_sampling_mask.sum()/var_sampling_mask.size)
print("Mask type:",  var_sampling_mask.dtype)
plt.figure()
plt.imshow(~var_sampling_mask,cmap = "gray")
plt.axis("off")
plt.show()

## Load Train Data

In [ ]:
# Get number of samples
ntrain = 0
for ii in range(len(kspace_files_train)):
    f = BytesIO(file_io.read_file_to_string(kspace_files_train[ii], binary_mode=True))
    ntrain += np.load(f).shape[0]
    
    print(ntrain)
    # ntrain += np.load(kspace_files_train[ii]).shape[0]
#     ntrain += 168

# print(ntrain)

# Load train data    
rec_train = np.zeros((ntrain,imshape[0],imshape[1],2))
kspace_train = np.zeros((ntrain,imshape[0],imshape[1],2))
aux_counter = 0
for ii in range(len(kspace_files_train)):
    f = BytesIO(file_io.read_file_to_string(kspace_files_train[ii], binary_mode=True))
    aux_kspace = np.load(f)/norm
    # aux_kspace = np.load(kspace_files_train[ii])/norm
    aux = aux_kspace.shape[0]   
    aux2 = np.fft.ifft2(aux_kspace[:,:,:,0]+1j*aux_kspace[:,:,:,1])
    rec_train[aux_counter:aux_counter+aux,:,:,0] = aux2.real
    rec_train[aux_counter:aux_counter+aux,:,:,1] = aux2.imag
    kspace_train[aux_counter:aux_counter+aux,:,:,0] = aux_kspace[:,:,:,0]
    kspace_train[aux_counter:aux_counter+aux,:,:,1] = aux_kspace[:,:,:,1]
    
    print(aux)
    aux_counter+=aux

# Shuffle training    
# indexes = np.arange(rec_train.shape[0],dtype = int)
# np.random.shuffle(indexes)
# rec_train = rec_train[indexes]

kspace_train[:,var_sampling_mask,:] = 0 # undersample k-space

# save k-space and image domain stats
stats = np.zeros(4)
stats[0] = kspace_train.mean()
stats[1] = kspace_train.std()
aux = np.abs(rec_train[:,:,:,0] +1j*rec_train[:,:,:,1])
stats[2] = aux.mean()
stats[3] = aux.std()
np.save("../Data/stats_fs_unet_norm_" + under_rate + ".npy",stats)

print("Number of training samples", rec_train.shape[0])
kspace_train = 0 # release memory

## Load Validation Data

In [ ]:
# Get number of samples
nval = 0
for ii in range(len(kspace_files_val)):
    f = BytesIO(file_io.read_file_to_string(kspace_files_val[ii], binary_mode=True))
    nval += np.load(f).shape[0]

kspace_val = np.zeros((nval,imshape[0],imshape[1],nchannels))
rec_val = np.zeros((nval,imshape[0],imshape[1],1))
aux_counter = 0
for ii in range(len(kspace_files_val)):
    f = BytesIO(file_io.read_file_to_string(kspace_files_val[ii], binary_mode=True))
    aux_kspace = np.load(f)/norm
    aux = aux_kspace.shape[0]   
    kspace_val[aux_counter:aux_counter+aux] = aux_kspace
    rec_val[aux_counter:aux_counter+aux,:,:,0] = \
    np.abs(np.fft.ifft2(aux_kspace[:,:,:,0]+1j*aux_kspace[:,:,:,1]))
    aux_counter+=aux

# Undersampling kspace
kspace_val2 = kspace_val.copy()
kspace_val[:,var_sampling_mask,:] = 0

rec_val = np.fft.ifft2(kspace_val[:,:,:,0]+1j*kspace_val[:,:,:,1])
rec_val2 = np.fft.ifft2(kspace_val2[:,:,:,0]+1j*kspace_val2[:,:,:,1])
rec_val = np.expand_dims(rec_val, axis=-1)

kspace_val = (kspace_val-stats[0])/stats[1]

print("Number of samples", kspace_val.shape[0])
print("Kspace under stats", kspace_val.mean(),kspace_val.std())
print("Kspace full stats", kspace_val2.mean(),kspace_val2.std())
print("Rec stats", rec_val.mean(),rec_val.std())

## Initialize Model

In [ ]:
epochs = 75
batch_size= 16 # changing batch size from 16 to 3 for data augmentation
model = fsnet.model1()
opt = Adam(lr=1e-3,decay = 1e-7)
model.compile(loss = fsnet.nrmse,optimizer=opt, metrics=['accuracy'])

model_name = "../Models/wnet_part1.hdf5"
# model_name = r"C:\Users\soggy\Videos\Hybrid-CS-Model-MRI\Models\wnet_part1.hdf5"
if os.path.isfile(model_name):
    model.load_weights(model_name)

print(model.summary())

# Early stopping callback to shut down training after
#10 epochs with no improvement
earlyStopping = EarlyStopping(monitor='val_loss',
                                       patience=20, 
                                       verbose=0, mode='min')

# Checkpoint callback to save model  along the epochs
checkpoint = ModelCheckpoint(model_name, mode = 'min', \
                             monitor='val_loss',verbose=0,\
                             save_best_only=True, save_weights_only = True)

## Data Augmentation

In [ ]:
seed = 905
image_datagen1 = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.075,
        height_shift_range=0.075,
        shear_range=0.25,
        zoom_range=0.25,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

image_datagen2 = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.075,
        height_shift_range=0.075,
        shear_range=0.25,
        zoom_range=0.25,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

image_datagen1.fit(rec_train[:,:,:,0,np.newaxis], augment=True, seed=seed)
image_datagen2.fit(rec_train[:,:,:,1,np.newaxis], augment=True, seed=seed)

image_generator1 = image_datagen1.flow(rec_train[:,:,:,0,np.newaxis],batch_size = batch_size,seed = seed)
image_generator2 = image_datagen1.flow(rec_train[:,:,:,1,np.newaxis],batch_size = batch_size,seed = seed)

def combine_generator(gen1,gen2,under_mask,stats):
    while True:
        rec_real = gen1.next()
        rec_imag = gen2.next()
        kspace = np.fft.fft2(rec_real[:,:,:,0]+1j*rec_imag[:,:,:,0])
        kspace2 = np.zeros((kspace.shape[0],kspace.shape[1],kspace.shape[2],2))
        kspace2[:,:,:,0] = kspace.real
        kspace2[:,:,:,1] = kspace.imag
        kspace_under = kspace2.copy()
        kspace_under[:,var_sampling_mask,:] = 0
        kspace_under = (kspace_under-stats[0])/stats[1]
        rec = np.abs(rec_real[:,:,:,0]+1j*rec_imag[:,:,:,0])[:,:,:,np.newaxis]
        
        rec_under = np.fft.ifft2(kspace_under[:,:,:,0]+1j*kspace_under[:,:,:,1])
        rec_under_expanded = np.expand_dims(rec_under, axis=-1)
#         print(rec_under.shape, rec_under_expanded.shape)
        yield(kspace_under, [kspace2])

# combine generators into one which yields image and masks
combined = combine_generator(image_generator1,image_generator2, var_sampling_mask,stats)

# sample data augmentation
# for ii in combined:
#     print(ii[1][1][1].shape)
#     plt.figure()
#     plt.subplot(221)
#     plt.imshow(ii[1][1][1][:,:,0],cmap = 'gray')
#     plt.axis("off")
#     plt.subplot(222)
#     plt.imshow(ii[1][1][1][:,:,0],cmap = 'gray')
#     plt.axis("off")
#     plt.subplot(223)
#     plt.imshow(ii[1][1][1][:,:,0],cmap = 'gray')
#     plt.axis("off")
#     plt.subplot(224)
#     plt.imshow(np.log(1+np.abs(ii[1][0][1][1,:,:,0] + 1j*ii[1][0][1][1,:,:,1])),cmap = 'gray')
#     plt.axis("off")
#     plt.show()
#     print(ii[1][0].shape)
#     break

## Train model

In [ ]:
hist = model.fit_generator(combined,
                 epochs=epochs,
#                  steps_per_epoch=rec_train.shape[0] / batch_size,
                 steps_per_epoch=rec_train.shape[0] / 16,
                 verbose=1,
                 validation_data= (kspace_val,[kspace_val2]),
                 callbacks=[checkpoint,earlyStopping])

In [ ]:
print(hist.history)

In [ ]:
dict = hist.history
import matplotlib.pyplot as plt

plt.plot(dict['loss'], color='b', label='train')
plt.plot(dict['val_loss'], color='r', label='val')
plt.title('Loss curves')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
# model.save(r"C:\Users\soggy\Videos\Hybrid-CS-Model-MRI\Models\wnet_part1.hdf5")